In [2]:
import re
import numpy as np
import itertools

In [3]:
def read_file(filename):
    with open(filename, 'r') as file:
        content = file.read()  # Open the file in read mode

    data = {}  # Initialize data as an empty dictionary

    # Each file has number of inputs under which data is stored as a matrix
    numerical_data = re.findall(r'\[([\d\s.]+)\]', content)  # Extract values following each header as numerical values and remove '\t's and '\n's

    sections = ['Orders', 'Allocations', 'DistanceShelfShelf', 'DistancePackagingShelf', 'FullDistanceMatrix']

    for i, section in enumerate(sections):
        data[section] = []
        lines = numerical_data[i].strip().split('\n')
        for line in lines:
            values = line.strip().split()  # Split using spaces
            data[section].append([int(val) for val in values])  # Assuming all values are integers

    return data

# Reading in data and running the entire function

In [4]:
filename = r"Data_Xpress_FullDist.txt"
data = read_file(filename)

# Extracting data into arrays
Orders = data.get('Orders')
Allocations = np.asarray(data.get('Allocations')[0])
DistanceShelfShelf = np.asarray(data.get('DistanceShelfShelf'))
FullDistanceMatrix = np.asarray(data.get('FullDistanceMatrix'))

NbShelves = 96
Shelves = range(1, NbShelves + 1)

FullDistanceMatrix = np.roll(FullDistanceMatrix, shift = 1, axis = 1)
FullDistanceMatrix = np.roll(FullDistanceMatrix, shift = 1, axis = 0)


# testing for allocation vector where every shelf is filled
allocations_full = [45, 79, 39, 68, 73, 53, 19, 44, 16, 71, 27, 41,  2, 46, 60, 67, 56, 
                    83, 80, 57, 69, 55, 75, 34, 89, 12, 81, 62, 23, 26, 24, 86,  3, 17,
                    90, 58, 51, 25, 85, 65, 31, 11, 87, 10, 13, 70, 35, 32, 47,  6, 30,
                    21, 43, 64, 66, 78, 76, 61,  8, 72, 22, 18, 82, 14, 28,  4,  5, 84,
                    54, 48, 63, 29, 49, 74, 37, 36, 20, 38, 50,  7, 88,  9, 40, 77, 15,
                    1, 33, 59, 42, 52,  45, 79, 39, 68, 73, 53]


# testing where only some of the extra shelves are filled
allocations_mid_full = [45, 79, 39, 68, 73, 53, 19, 44, 16, 71, 27, 41,  2, 46, 60, 67, 56, 
                        83, 80, 57, 69, 55, 75, 34, 89, 12, 81, 62, 23, 26, 24, 86,  3, 17,
                        90, 58, 51, 25, 85, 65, 31, 11, 87, 10, 13, 70, 35, 32, 47,  6, 30,
                        21, 43, 64, 66, 78, 76, 61,  8, 72, 22, 18, 82, 14, 28,  4,  5, 84,
                        54, 48, 63, 29, 49, 74, 37, 36, 20, 38, 50,  7, 88,  9, 40, 77, 15,
                        1, 33, 59, 42, 52,  45, 79,  0, 68,  0, 53]


In [5]:
Allocations

array([45, 79, 39, 68, 73, 53, 19, 44, 16, 71, 27, 41,  2, 46, 60, 67, 56,
       83, 80, 57, 69, 55, 75, 34, 89, 12, 81, 62, 23, 26, 24, 86,  3, 17,
       90, 58, 51, 25, 85, 65, 31, 11, 87, 10, 13, 70, 35, 32, 47,  6, 30,
       21, 43, 64, 66, 78, 76, 61,  8, 72, 22, 18, 82, 14, 28,  4,  5, 84,
       54, 48, 63, 29, 49, 74, 37, 36, 20, 38, 50,  7, 88,  9, 40, 77, 15,
        1, 33, 59, 42, 52,  0,  0,  0,  0,  0,  0])

In [6]:
def q1_function(allocation_vector):

    def find_closest_product(current_position, products, distances):
        closest_product = None
        min_distance = float('inf')
        for product in products:
            if distances[current_position][product] <= min_distance:
                closest_product = product
                min_distance = distances[current_position][product]
        return closest_product

    def generate_order_lists(order):
        ''' 
        Takes a list with tuple elements and returns a list of lists 
        with all possible combinations of individual elements individual elements.
        '''
        order_lists = []
        tuple_indices = [i for i, item in enumerate(order) if isinstance(item, tuple)]
        for combination in itertools.product(*[order[i] for i in tuple_indices]):
            new_order = order.copy()
            for i, index in enumerate(tuple_indices):
                new_order[index] = combination[i]
            order_lists.append(new_order)
        return order_lists

    def greedy_order_route(order, distances):
        ''' 
        Uses a greedy method of calculating the minimum distance. 

        Function has been split into two if statements to consider
        cases of orders where products are contained on more than one shelf.

        If products are contained on more than one shelf, the function constructs 
        a route with all possible shelf combinations and chooses the one with 
        the shortest distance.

        Returns a list containing the route and the total distance for the order.
        '''

        # If all products in the order are contained on one shelf only
        if not any(isinstance(product, tuple) for product in order):
            visited = [0]
            current_position = 0  
            for k in range(len(order)):
                    closest_product = find_closest_product(current_position, order, distances)
                    visited.append(closest_product)
                    order.remove(closest_product)
                    current_position = closest_product
            visited.append(0)
            OrderDistance = 0
            for i in range(len(visited) - 1):
                OrderDistance += distances[visited[i]][visited[i+1]]
            order_distance_final = OrderDistance
            visited_final = visited

        # If one or more products in the order are contained on more than one shelf
        elif any(isinstance(product, tuple) for product in order):
            order_combinations = generate_order_lists(order) # create new orders with all possible combinations from tuples
            order_routes = []   # initialise a list of routes for all combinations                                 
            order_distances = [] # initialise a list of distances for all combinations
            
            # loop over all combinations
            for order in order_combinations:
                visited = [0]
                current_position = 0  
                for k in range(len(order)):
                    closest_product = find_closest_product(current_position, order, distances)
                    visited.append(closest_product)
                    order.remove(closest_product)
                    current_position = closest_product
                visited.append(0)
                order_routes.append(visited) # add the route for the combination to the list of routes
                OrderDistance = 0
                for i in range(len(visited) - 1):
                    OrderDistance += distances[visited[i]][visited[i+1]]
                order_distances.append(OrderDistance) # add the distance for the combination to the list of distances
            
            # select the order with the shortest distance among the combinations
            min_idx = order_distances.index(min(order_distances))
            visited_final = order_routes[min_idx] 
            order_distance_final = order_distances[min_idx]
                
        return visited_final, order_distance_final # return order route and distance
    
    def convert_orders_to_shelf_indices(allocations):
        ''' 
        This function takes the allocation vector and returns an 
        order matrix with shelf indices instead of product indices.
        '''
        product_to_shelf = {}
        for shelf_index, product in enumerate(allocations):
            if product != 0:  # Check if the element is not zero
                if product not in product_to_shelf:
                    product_to_shelf[product] = [shelf_index + 1]  # Initialize with a list containing the current shelf index
                else:
                    # If the product already exists in the dictionary, append the new shelf index to the list
                    product_to_shelf[product].append(shelf_index + 1)

        # Convert product_to_shelf dictionary to a list of tuples if the product is assigned to multiple shelves
        product_to_shelf_tuples = {k: tuple(v) if len(v) > 1 else v[0] for k, v in product_to_shelf.items()}

        OrdersByShelf = []
        for order in Orders:
            order_shelf_indices = []
            for product_index in order:
                if product_index in product_to_shelf_tuples:
                    shelf_indices = product_to_shelf_tuples[product_index]
                    order_shelf_indices.append(shelf_indices)
                else:
                    order_shelf_indices.append(0)  # Product not found in allocation matrix
            OrdersByShelf.append(order_shelf_indices)

        return OrdersByShelf
    

    OrdersByShelf = convert_orders_to_shelf_indices(allocation_vector)
    TotalDistance = 0           # initialise counter for total distance
    DistancesPerOrder = []      # initalise list to contain the distances for each order 
    routes = []                 # intialise list to contain the routes for each order
    for order in OrdersByShelf:
        visited_order_route, visited_order_dist = greedy_order_route(order, FullDistanceMatrix)
        routes.append(visited_order_route)
        DistancesPerOrder.append(3*visited_order_dist)
        TotalDistance += 3*visited_order_dist

    # Replace DistancesPerOrder in the return statement with this if you want sorted distances to be returned
    SortedDistancesPerOrder = sorted(DistancesPerOrder, reverse=True)

    # The indices corresponding to the longest orders (descending order) 
    idx_longest_orders = sorted(range(len(DistancesPerOrder)), key=lambda i: DistancesPerOrder[i], reverse=True)
    
    return TotalDistance, DistancesPerOrder, SortedDistancesPerOrder, idx_longest_orders, routes

In [47]:
### TEST CASE ON CURRENT ALLOCATION ###
TotalDistance, DistancesPerOrder, SortedDistancesPerOrder, idx_longest_orders, routes = q1_function(Allocations)

In [48]:
print(routes)

[[0, 0, 0, 0, 51, 79, 0], [0, 0, 0, 73, 57, 62, 0], [0, 12, 37, 47, 90, 60, 0], [0, 0, 0, 0, 79, 66, 0], [0, 50, 57, 30, 7, 19, 0], [0, 55, 68, 16, 37, 10, 0], [0, 73, 3, 18, 72, 62, 0], [0, 0, 0, 0, 81, 13, 0], [0, 0, 0, 0, 7, 71, 0], [0, 59, 84, 65, 42, 41, 0], [0, 78, 56, 8, 41, 48, 0], [0, 51, 4, 41, 44, 71, 0], [0, 0, 81, 47, 46, 13, 0], [0, 56, 84, 24, 46, 10, 0], [0, 0, 78, 59, 85, 35, 0], [0, 0, 0, 50, 61, 71, 0], [0, 0, 0, 78, 72, 13, 0], [0, 0, 0, 0, 85, 88, 0], [0, 25, 3, 37, 24, 59, 0], [0, 10, 41, 46, 23, 70, 0], [0, 0, 0, 0, 10, 44, 0], [0, 0, 0, 0, 74, 85, 0], [0, 0, 0, 0, 8, 85, 0], [0, 0, 0, 0, 0, 37, 0], [0, 0, 7, 16, 40, 79, 0], [0, 10, 16, 70, 67, 81, 0], [0, 0, 50, 51, 3, 41, 0], [0, 0, 0, 0, 0, 45, 0], [0, 0, 0, 0, 0, 36, 0], [0, 0, 0, 0, 0, 50, 0], [0, 0, 0, 0, 70, 71, 0], [0, 0, 13, 46, 62, 85, 0], [0, 0, 0, 0, 50, 54, 0], [0, 78, 62, 63, 47, 18, 0], [0, 0, 0, 2, 59, 71, 0], [0, 49, 60, 22, 36, 8, 0], [0, 0, 0, 0, 0, 65, 0], [0, 0, 73, 9, 44, 22, 0], [0, 0, 0, 5

#### Obtaining total distances for different allocations

In [49]:
print(f"Distance for the current allocation: {q1_function(Allocations)[0]} metres.")
print(f"Distance for all shelves filled: {q1_function(allocations_full)[0]} metres.")
print(f"Distance for some repeated products and some empty shelves: {q1_function(allocations_mid_full)[0]} metres.")

Distance for the current allocation: 290706 metres.
Distance for all shelves filled: 289200 metres.
Distance for some repeated products and some empty shelves: 289728 metres.


In [62]:
for i, order in enumerate(Orders):
    print(f"Order is {order}, route taken is {routes[i]}")

Order is [50, 30, 0, 0, 0], route taken is [0, 0, 0, 0, 51, 79, 0]
Order is [49, 18, 76, 0, 0], route taken is [0, 0, 0, 73, 57, 62, 0]
Order is [72, 52, 51, 41, 35], route taken is [0, 12, 37, 47, 90, 60, 0]
Order is [50, 4, 0, 0, 0], route taken is [0, 0, 0, 0, 79, 66, 0]
Order is [76, 19, 26, 80, 6], route taken is [0, 50, 57, 30, 7, 19, 0]
Order is [66, 67, 51, 84, 71], route taken is [0, 55, 68, 16, 37, 10, 0]
Order is [39, 18, 29, 83, 49], route taken is [0, 73, 3, 18, 72, 62, 0]
Order is [88, 2, 0, 0, 0], route taken is [0, 0, 0, 0, 81, 13, 0]
Order is [63, 19, 0, 0, 0], route taken is [0, 0, 0, 0, 7, 71, 0]
Order is [28, 11, 77, 8, 31], route taken is [0, 59, 84, 65, 42, 41, 0]
Order is [32, 78, 38, 31, 44], route taken is [0, 78, 56, 8, 41, 48, 0]
Order is [63, 31, 10, 68, 30], route taken is [0, 51, 4, 41, 44, 71, 0]
Order is [88, 70, 35, 2, 0], route taken is [0, 0, 81, 47, 46, 13, 0]
Order is [71, 70, 77, 34, 78], route taken is [0, 56, 84, 24, 46, 10, 0]
Order is [38, 8, 1

In [60]:
def BruteForceMinDist(order, shelves, dist):
    shortest_path = None
    shortest_dist = 0


    order = [num for num in order if num != 0]

    totaldist = 1000000000000
    route = []


    permutations = itertools.permutations(order)

    for perm in permutations:
        shelf_nums = []
        for value in perm:
            index = shelves.index(value)
            shelf_nums.append(index)

        tempdist = dist[96, shelf_nums[0]]


        

        if len(perm)>1:
            tempdist += dist[shelf_nums[0], shelf_nums[1]]
        if len(perm)>2:
            tempdist += dist[shelf_nums[1], shelf_nums[2]]
        if len(perm)>3:
            tempdist += dist[shelf_nums[2], shelf_nums[3]]
        if len(perm)>4:
            tempdist += dist[shelf_nums[3], shelf_nums[4]] 

        tempdist += dist[shelf_nums[-1], 96]

        if tempdist < totaldist:
            totaldist = tempdist
            route = shelf_nums
        


    return totaldist, route


def BF_ObjectiveFunction(orders_list, shelves, distances, dummy = True):
    totaldist = 0
    routes = []
    individualdists = []

    for i, order in enumerate(orders_list):
        dist, route = BruteForceMinDist(order, shelves, distances)
        totaldist += dist
        routes.append(route)
        individualdists.append(dist)
    return totaldist, routes, individualdists


Tests

In [61]:
print(len(Allocations))
print(Allocations)

test, BFroutes, orderdists = BF_ObjectiveFunction(Orders, list(Allocations), FullDistanceMatrix)

print(test)
print(Orders)
print(BFroutes)
print(routes)
print(orderdists)
print(DistancesPerOrder)

96
[45 79 39 68 73 53 19 44 16 71 27 41  2 46 60 67 56 83 80 57 69 55 75 34
 89 12 81 62 23 26 24 86  3 17 90 58 51 25 85 65 31 11 87 10 13 70 35 32
 47  6 30 21 43 64 66 78 76 61  8 72 22 18 82 14 28  4  5 84 54 48 63 29
 49 74 37 36 20 38 50  7 88  9 40 77 15  1 33 59 42 52  0  0  0  0  0  0]
92596
[[50, 30, 0, 0, 0], [49, 18, 76, 0, 0], [72, 52, 51, 41, 35], [50, 4, 0, 0, 0], [76, 19, 26, 80, 6], [66, 67, 51, 84, 71], [39, 18, 29, 83, 49], [88, 2, 0, 0, 0], [63, 19, 0, 0, 0], [28, 11, 77, 8, 31], [32, 78, 38, 31, 44], [63, 31, 10, 68, 30], [88, 70, 35, 2, 0], [71, 70, 77, 34, 78], [38, 8, 15, 90, 0], [6, 22, 63, 0, 0], [29, 2, 38, 0, 0], [15, 59, 0, 0, 0], [8, 39, 51, 34, 89], [75, 31, 48, 70, 71], [10, 71, 0, 0, 0], [74, 15, 0, 0, 0], [44, 15, 0, 0, 0], [51, 0, 0, 0, 0], [19, 65, 50, 67, 0], [48, 5, 88, 71, 67], [30, 39, 6, 31, 0], [13, 0, 0, 0, 0], [58, 0, 0, 0, 0], [6, 0, 0, 0, 0], [63, 48, 0, 0, 0], [18, 15, 70, 2, 0], [64, 6, 0, 0, 0], [83, 82, 18, 35, 38], [63, 79, 8, 0, 0], [

In [59]:
print(FullDistanceMatrix)

[[ 0  2  3 ... 24 25 26]
 [ 2  0  1 ... 26 27 28]
 [ 3  1  0 ... 27 28 27]
 ...
 [24 26 27 ...  0  1  2]
 [25 27 28 ...  1  0  1]
 [26 28 27 ...  2  1  0]]
